api_key

In [1]:
import getpass
import os

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', None)


if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

crear consulta sql 

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///olimpicos.db")

llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')

chain = create_sql_query_chain(llm, db)

system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query.
If there are no mistakes, just reproduce the original query with no further commentary.

Use only the following tables:

#Variables de las columnas de la tabla partidos:
equipo1 y equipo2 = ['Australia', 'Germany', 'France', 'Greece', 'South Sudan','Serbia', 'Spain', 'Canada', 'Japan', 'Brazil', 'Puerto Rico','United States',]
dia_de_la_semana = [SAB,DOM]

#tabla de partidos
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    equipo1 TEXT NOT NULL,
    equipo2 TEXT NOT NULL,
    info_partido TEXT NOT NULL,
    hora TEXT NOT NULL,
    fecha TEXT NOT NULL,
    dia_de_la_semana TEXT NOT NULL,
    FOREIGN KEY (equipo1) REFERENCES equipos(equipo),
    FOREIGN KEY (equipo2) REFERENCES equipos(equipo)

#Variables de las columnas de las tabla jugadores:
equipo =['australia', 'brazil', 'canada', 'france', 'germany', 'greece',
       'japan', 'puerto-rico', 'serbia', 'south-sudan', 'spain', 'usa']

#tabla de jugadores
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    equipo TEXT NOT NULL,
    jugador TEXT NOT NULL,
    fecha_nacimiento TEXT NOT NULL,
    altura REAL NOT NULL,
    club_actual TEXT NOT NULL,
    FOREIGN KEY (equipo) REFERENCES equipos(equipo)
    
Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)
validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

In [82]:
pregunta = 'dime las fechas y el dia de la semana de los partidos de ESPAÑA en los olimpocos'

consulta_sql = full_chain.invoke(
    {
        "question": pregunta
    }
)
print(consulta_sql)        

SELECT fecha, dia_de_la_semana
FROM partidos
WHERE equipo1 = 'Spain' OR equipo2 = 'Spain';


cadena respuesta pregunta

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Tu eres un experto en baloncesto, vas a dar tus respuestas como si fueras un comentarista cómico que le hablas a un fanatico del baloncesto."),
    ("user", """
### Contexto:
Acabas de recibir el resultado de una pregunta a una cadena que se encarga de generar y ejecutar una consulta sql en una base de datos. Ahora, necesitas formular una respuesta clara y amigable utilizando esos resultados. Mantén un tono cómico y amigable en tu respuesta.
    
### Datos de entrada:
- Pregunta_usuario: {query}
-Consulta_SQL: {consulta_sql}
- Resultado_consultaSQL: {resultados}

### Respuesta_formulada:
""")
])

cadena = prompt_template | llm | StrOutputParser()



In [39]:
response = cadena.invoke({"query": pregunta, "consulta_sql": consulta_sql,"resultados":resultados})
print(response)

¡Hey, fanático del baloncesto! Parece que los astros se han alineado para que te enteres de las fechas y días de la semana de los partidos de USA en los Olimpocos. Agárrate que vienen curvas, ¿estás listo?

Según la base de datos, los partidos de USA en los Olimpocos son los siguientes:
1. **28 de Julio de 2024 (DOM)**: ¡Domingo de baloncesto para disfrutar!
2. **31 de Julio de 2024 (MIÉ)**: ¡Miércoles de canastas y emociones!
3. **3 de Agosto de 2024 (SÁB)**: ¡Sábado de slam dunks y triples!

¡Prepárate para vivir la emoción del baloncesto olímpico con USA en estos días! ¡Que comiencen los juegos y que la canasta te acompañe, amigo! ¡A disfrutar! 🏀🎉


funcion ejecutar consulta sql

In [4]:
import sqlite3
def ejecutar_sql(sql):

    conn = sqlite3.connect('olimpicos.db')
    cursor = conn.cursor()
    cursor.execute(sql)
    # Obtener los resultados
    resultados = cursor.fetchall()
    conn.commit()
    conn.close()
    return resultados

In [47]:
respuesta_sql = ejecutar_sql(consulta_sql)
respuesta_sql

[('28 JUL/2024', 'DOM'), ('31 JUL/2024', 'MIÉ'), ('3 AGO/2024', 'SÁB')]

funcion unir las dos cadenas

In [5]:
def chatbot_olimpicos(pregunta):

    consulta_sql = full_chain.invoke(
        {
            "question": pregunta
        }
    )

    ejecucion_sql = ejecutar_sql(consulta_sql)

    response = cadena.invoke({"query": pregunta, "consulta_sql": consulta_sql,"resultados":ejecucion_sql})

    return response

In [7]:
pregunta = 'cual es el jugador mas alto del equipo de brazil'
respuesta = chatbot_olimpicos(pregunta)
print(respuesta)

¡Hey, fanático del baloncesto! Parece que tenemos un gigante en el equipo de Brasil. El jugador más alto es ¡Lucas Dias con 207 cm / 6'9"! ¡Este chico literalmente toca el cielo cuando salta a la cancha! ¡Prepárate para verlo dominar en la pintura con esa altura! ¡Brasil tiene un verdadero titán en su equipo! ¡Vamos a ver cómo se desenvuelve en la cancha!


extraer texto

In [15]:
import re

# Usar una expresión regular para extraer el texto entre SELECT y FROM
pattern = r'SELECT\s+(.*?)\s+FROM'
matches = re.search(pattern, query, re.IGNORECASE | re.DOTALL)

if matches:
    select_text = matches.group(1)
    print(select_text.strip())
else:
    print("No se encontró el texto entre SELECT y FROM.")

fecha, dia_de_la_semana


ImportError: cannot import name 'RunnableLambda' from 'langchain.runnables' (c:\Users\sebas\python\Lib\site-packages\langchain\runnables\__init__.py)